In [1]:
import pandas as pd

ruta = r"C:\Users\Usuario\Desktop\SIC\SEGURIDAD.xlsx"
df = pd.read_excel(ruta)

print(df.columns)


Index(['Id', 'Start time', 'Completion time', 'Email', 'Name', 'Latitud',
       'Longitud',
       'Nivel de Iluminación\n\nEn una escala del 0 al 1\n1 = Excelente\n0 = Muy mala\n\nDescriba la cantidad de iluminación de la zona',
       'Cantidad de comercios cercanos',
       'Indique el número de reportes de incidentes en la zona',
       'Flujo de personas\n\nDel 1 al 5\n\n1 = Casi nadie\n5 = Muy concurrido\n\nEscriba según el flujo de personas que correspondan',
       'Distancia al punto de policía más cercano (en metros)', 'Comentarios'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import folium
from folium.plugins import MarkerCluster

# --------------------------------------
# 1. CARGAR ARCHIVO LOCAL
# --------------------------------------

ruta = r"C:\Users\Usuario\Desktop\SIC\SEGURIDAD.xlsx"  # ← cambia si es necesario
df = pd.read_excel(ruta)

print("Archivo cargado correctamente\n")
print(df.head())

# --------------------------------------
# 2. RENOMBRAR COLUMNAS LARGAS A CORTAS
# --------------------------------------

df = df.rename(columns={
    "Latitud": "latitud",
    "Longitud": "longitud",
    "Nivel de Iluminación\n\nEn una escala del 0 al 1\n1 = Excelente\n0 = Muy mala\n\nDescriba la cantidad de iluminación de la zona": "iluminacion",
    "Cantidad de comercios cercanos": "comercios",
    "Indique el número de reportes de incidentes en la zona": "reportes",
    "Flujo de personas\n\nDel 1 al 5\n\n1 = Casi nadie\n5 = Muy concurrido\n\nEscriba según el flujo de personas que correspondan": "flujo",
    "Distancia al punto de policía más cercano (en metros)": "distancia_policia"
})

# --------------------------------------
# 3. PREPARAR DATOS
# --------------------------------------

features = ["iluminacion", "comercios", "reportes", "flujo", "distancia_policia"]

df[features] = df[features].apply(pd.to_numeric, errors="coerce")

df = df.dropna(subset=features + ["latitud", "longitud"])

# --------------------------------------
# 4. MODELO K-MEANS
# --------------------------------------

kmeans = KMeans(n_clusters=3, random_state=0)
df["cluster"] = kmeans.fit_predict(df[features])

# --------------------------------------
# 5. COLORES
# --------------------------------------

colores = {
    0: "green",
    1: "orange",
    2: "red"
}

df["color"] = df["cluster"].map(colores)

# --------------------------------------
# 6. MAPA INTERACTIVO
# --------------------------------------

centro = [df["latitud"].mean(), df["longitud"].mean()]
mapa = folium.Map(location=centro, zoom_start=14)
marker_cluster = MarkerCluster().add_to(mapa)

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["latitud"], row["longitud"]],
        radius=40,                # ¡CÍRCULO MÁS GRANDE!
        color=row["color"],
        fill=True,
        fill_color=row["color"],
        fill_opacity=0.6,         # opcional: mejor visibilidad
        popup=f"Cluster: {row['cluster']}"
    ).add_to(marker_cluster)

mapa.save("mapa_zonas.html")

print("\n[OK] Mapa generado exitosamente: mapa_zonas.html")



Archivo cargado correctamente

   Id          Start time     Completion time      Email  Name   Latitud  \
0   1 2025-12-10 17:42:21 2025-12-10 17:45:26  anonymous   NaN  0.417770   
1   2 2025-12-10 17:45:33 2025-12-10 17:46:30  anonymous   NaN  0.419034   
2   3 2025-12-10 17:46:44 2025-12-10 17:47:26  anonymous   NaN  0.419064   
3   4 2025-12-10 17:47:51 2025-12-10 17:49:20  anonymous   NaN  0.417030   
4   5 2025-12-10 17:49:23 2025-12-10 17:50:54  anonymous   NaN  0.417031   

    Longitud  \
0 -78.192835   
1 -78.192658   
2 -78.191593   
3 -78.192886   
4 -78.189421   

   Nivel de Iluminación\n\nEn una escala del 0 al 1\n1 = Excelente\n0 = Muy mala\n\nDescriba la cantidad de iluminación de la zona  \
0                                                1.0                                                                                 
1                                                0.0                                                                                 
2            

c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
